In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from sim_class import Simulation

# Create the class
class OT2Env(gym.Env):
    def __init__(self, render=True, max_steps=1000):
        super(OT2Env, self).__init__()
        self.render = render
        self.max_steps = max_steps

        # Create the simulation environment
        self.sim = Simulation(num_agents=1, render=self.render)

        # Set the maximum values according to the working environment.
        self.x_min, self.x_max = -0.187, 0.2531
        self.y_min, self.y_max = -0.1705, 0.2195
        self.z_min, self.z_max = 0.1195, 0.2895
        # Define action and observation spaces
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(3,), dtype=np.float32)
        self.observation_space = spaces.Box(
            low=np.array([self.x_min, self.y_min, self.z_min, -self.x_max, -self.y_max, -self.z_max], dtype=np.float32),
            high=np.array([self.x_max, self.y_max, self.z_max, self.x_max, self.y_max, self.z_max], dtype=np.float32),
            dtype=np.float32
        )

        # Keep track of the step amount
        self.steps = 0

    def reset(self, seed=None):
        # Set a seed if it was not set yet
        if seed is not None:
            np.random.seed(seed)

        # Randomise the goal position
        x = np.random.uniform(self.x_min, self.x_max)
        y = np.random.uniform(self.y_min, self.y_max)
        z = np.random.uniform(self.z_min, self.z_max)
        # Set a random goal position
        self.goal_position = np.array([x, y, z])
        # Call reset function
        observation = self.sim.reset(num_agents=1)
        # Set the observation.
        observation = np.concatenate(
            (
                self.sim.get_pipette_position(self.sim.robotIds[0]), 
                self.goal_position
            ), axis=0
        ).astype(np.float32) 

        # Reset the number of steps
        self.steps = 0

        info = {}

        return observation, info

    def step(self, action):
        # set the actions
        action = np.append(np.array(action, dtype=np.float32), 0)
        # Call the step function
        observation = self.sim.run([action])
        pipette_position = self.sim.get_pipette_position(self.sim.robotIds[0])
        # Process observation
        observation = np.array(pipette_position, dtype=np.float32)
        # Calculate the agent's reward
        distance = np.linalg.norm(np.array(pipette_position) - np.array(self.goal_position))
        reward = -distance
        
        # Check if the agent reaches within the threshold of the goal position
        if np.linalg.norm(pipette_position - self.goal_position) <= 0.001:
            terminated = True
        else:
            terminated = False

        # Check if episode should be truncated
        if self.steps >= self.max_steps:
            truncated = True
        else:
            truncated = False
        observation = np.concatenate((pipette_position, self.goal_position), axis=0).astype(np.float32)
        info = {}

        # Update the amount of steps
        self.steps += 1

        return observation, reward, terminated, truncated, info

    def render(self, mode='human'):
        pass
    
    def close(self):
        self.sim.close()

In [3]:
from stable_baselines3 import PPO  # Replace PPO with your RL algorithm
import numpy as np


# Path to the trained model
model_path = "C:\\Users\\jarro\\Documents\\GitHub\\Reinforcement learning\\model.zip"  # Replace with the actual path to your model

# Load the trained model
model = PPO.load(model_path)  # Replace PPO with the correct algorithm

# Initialize the environment
env = OT2Env(render=True)  # Initialize the environment with rendering disabled

# Test the trained model
def test_model(env, model, max_steps=1000):
    """
    Test the trained model to move the pipette to the target location.

    Args:
        env: The testing environment.
        model: The trained RL model.
        max_steps: Maximum number of steps for the test.

    Returns:
        None
    """
    # Reset the environment
    obs, _ = env.reset()

    print(f"Goal Position: {env.goal_position}")

    for step in range(max_steps):
        # Predict the action for the current state
        action, _ = model.predict(obs, deterministic=True)  # Use deterministic=True for testing
        obs, _, terminated, truncated, _ = env.step(action)  # Execute the action

        # Calculate the accuracy (distance to the goal) in mm
        current_position = obs[:3]  # Extract the current position from the observation
        goal_position = env.goal_position
        distance_to_goal = np.linalg.norm(goal_position - current_position) * 1000  # Convert to mm

        # Print debug information
        print(f"Step {step + 1}:")
        print(f"  Current Position: {current_position}")
        print(f"  Goal Position: {goal_position}")
        print(f"  Distance to Goal: {distance_to_goal:.2f} mm")

        # Check if the goal is reached or the environment signals termination
        if terminated or distance_to_goal <= 1.0:  # 1 mm accuracy requirement
            print("Goal Reached!" if distance_to_goal <= 1.0 else "Environment Terminated!")
            break

    # Close the environment
    env.close()


# Run the test
if __name__ == "__main__":
    test_model(env, model, max_steps=10000)


Goal Position: [0.23310898 0.06773573 0.18234673]
Step 1:
  Current Position: [0.07345802 0.09005769 0.12024033]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 172.75 mm
Step 2:
  Current Position: [0.07437398 0.09089912 0.12097898]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 171.75 mm
Step 3:
  Current Position: [0.07574775 0.09170712 0.12172438]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 170.33 mm
Step 4:
  Current Position: [0.07757922 0.09247853 0.12248037]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 168.48 mm
Step 5:
  Current Position: [0.07986826 0.09321105 0.12325046]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 166.20 mm
Step 6:
  Current Position: [0.0826147  0.0939016  0.12403858]
  Goal Position: [0.23310898 0.06773573 0.18234673]
  Distance to Goal: 163.50 mm
Step 7:
  Current Position: [0.08581839 0.09454855 0.12484747]
  Goal Position: 

error: Not connected to physics server.